# News dataset

In [3]:
%pip install google-cloud-bigquery pandas db-dtypes google-auth

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google.cloud import bigquery
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

# Parameters
YEAR = 2024
FINAL_EVENTS_FILE = f"gdelt_events_clean_{YEAR}.csv"
PROJECT_ID = os.getenv("PROJECT_ID", "")

# Event columns we need
COLUMN_NAMES = [
    "GLOBALEVENTID", "SQLDATE", "Actor1CountryCode", "ActionGeo_FullName",
    "ActionGeo_Lat", "ActionGeo_Long", "EventRootCode", "GoldsteinScale",
    "NumMentions", "NumSources", "AvgTone"
]

KEYWORDS = [
    "port", "harbour", "harbor", "terminal", "shipment", "logistics",
    "container", "freight", "cargo", "vessel", "maritime", "supply",
    "trade", "import", "export", "customs", "sanction", "blockade",
    "strike", "riot", "protest", "labor", "labour", "disruption",
    "storm", "hurricane", "typhoon", "flood", "earthquake", "covid",
    "pandemic", "lockdown", "weather", "cyclone"
]

print("=" * 80)
print("GDELT DATA DOWNLOAD VIA GOOGLE BIGQUERY")
print("=" * 80)

# Initialize BigQuery client (uses anonymous public access)
print("\n🔧 Initializing BigQuery client...")
os.environ["GCLOUD_PROJECT"] = PROJECT_ID
client = bigquery.Client(project=PROJECT_ID)

# Build keyword filter for SQL
keyword_conditions = " OR ".join([
    f"LOWER(ActionGeo_FullName) LIKE '%{kw.lower()}%'" 
    for kw in KEYWORDS
])

# BigQuery SQL to fetch maritime supply chain disruption events for 2020
query = f"""
SELECT 
    GLOBALEVENTID,
    SQLDATE,
    Actor1CountryCode,
    ActionGeo_FullName,
    ActionGeo_Lat,
    ActionGeo_Long,
    EventRootCode,
    GoldsteinScale,
    NumMentions,
    NumSources,
    AvgTone
FROM 
    `gdelt-bq.gdeltv2.events`
WHERE 
    SQLDATE >= {YEAR}0101 
    AND SQLDATE <= {YEAR}1231
    AND ActionGeo_FullName IS NOT NULL
    AND ({keyword_conditions})
    AND ActionGeo_Lat IS NOT NULL
    AND ActionGeo_Long IS NOT NULL
ORDER BY 
    SQLDATE
"""

print(f"\n📊 Querying GDELT BigQuery for {YEAR} maritime events...")
print(f"   Date range: {YEAR}-01-01 to {YEAR}-12-31")
print(f"   Filtering: {len(KEYWORDS)} supply chain keywords")
print(f"   This may take 2-5 minutes depending on data size...\n")

try:
    # Execute query
    query_job = client.query(query)
    
    # Fetch results with progress bar
    print("⏳ Executing query and fetching results...")
    events_df = query_job.to_dataframe(progress_bar_type='tqdm')
    
    if len(events_df) == 0:
        print("❌ No events found! Check your BigQuery access or filters.")
    else:
        # Data processing
        print(f"\n✅ Retrieved {len(events_df):,} events from BigQuery")
        
        # Convert date field
        events_df["Date"] = pd.to_datetime(events_df["SQLDATE"], format="%Y%m%d")
        
        # Convert numeric fields
        numeric_cols = ["ActionGeo_Lat", "ActionGeo_Long", "GoldsteinScale", 
                       "NumMentions", "NumSources", "AvgTone"]
        for col in numeric_cols:
            events_df[col] = pd.to_numeric(events_df[col], errors='coerce')
        
        # Remove rows with missing critical data
        events_df = events_df.dropna(subset=["ActionGeo_Lat", "ActionGeo_Long", "GoldsteinScale"])
        
        print(f"   After cleaning: {len(events_df):,} events")
        print(f"   Date range: {events_df['Date'].min()} to {events_df['Date'].max()}")
        print(f"   Countries: {events_df['Actor1CountryCode'].nunique()}")
        print(f"   Locations: {events_df['ActionGeo_FullName'].nunique()}")
        
        # Save cleaned events file
        events_df.to_csv(FINAL_EVENTS_FILE, index=False)
        print(f"\n✅ Saved to: {FINAL_EVENTS_FILE}")
        print(f"   File size: {os.path.getsize(FINAL_EVENTS_FILE) / (1024*1024):.2f} MB")
        
        # Display sample data
        print("\n📋 Sample events (first 5 rows):")
        print(events_df[["Date", "Actor1CountryCode", "ActionGeo_FullName", 
                        "GoldsteinScale", "NumMentions"]].head())
        
        print("\n" + "=" * 80)
        print("🎉 GDELT DATA DOWNLOAD COMPLETE!")
        print("=" * 80)
        print(f"\n📁 Output file:")
        print(f"   {FINAL_EVENTS_FILE} - Detailed event-level data")
        print(f"\n✅ Ready to merge with AIS data for model training!")

except Exception as e:
    print(f"\n❌ BigQuery Error: {e}")
    print("\n💡 TROUBLESHOOTING:")
    print("   1. Install package: pip install google-cloud-bigquery pandas db-dtypes")
    print("   2. BigQuery uses anonymous public access for GDELT")
    print("   3. If quota exceeded, try the synthetic data approach (option 3)")
    print("   4. Check internet connection")

GDELT DATA DOWNLOAD VIA GOOGLE BIGQUERY

🔧 Initializing BigQuery client...

📊 Querying GDELT BigQuery for 2024 maritime events...
   Date range: 2024-01-01 to 2024-12-31
   Filtering: 34 supply chain keywords
   This may take 2-5 minutes depending on data size...

⏳ Executing query and fetching results...
Job ID 042d3576-b830-431e-96ec-be6cf42198a9 successfully executed: 100%|██████████|

c:\Users\EE HANN\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|

✅ Retrieved 466,495 events from BigQuery
   After cleaning: 466,495 events
   Date range: 2024-01-01 00:00:00 to 2024-12-31 00:00:00
   Countries: 213
   Locations: 4427

✅ Saved to: gdelt_events_clean_2024.csv
   File size: 48.83 MB

📋 Sample events (first 5 rows):
        Date Actor1CountryCode                          ActionGeo_FullName  \
0 2024-01-01               AUS  Port Macquarie, New South Wales, Australia   
1 2024-01-01              None  Port Macquarie, New South Wales, Australia   
2 2024-01-01               AUS    Shellharbour, New South Wales, Australia   
3 2024-01-01              None    Port Hacking, New South Wales, Australia   
4 2024-01-01               USA             Portland, Oregon, United States   

   GoldsteinScale  NumMentions  
0             6.0            4  
1             6.0            4  
2             3.4            3  
3             3.0           10  
4            -4.4           10  

🎉 GDELT DATA DOWNLOAD COMPLETE!

